In [1]:
from dotenv import load_dotenv
import os
import json
import pandas as pd
from openai import OpenAI
import numpy as np
import yaml

In [2]:
load_dotenv()
OPEN_AI_API_KEY = os.getenv("OPEN_AI_API")

In [3]:
aita_df = df = pd.read_excel("AITA_Examples.xlsx") #.dropna(subset=['Raw reddit scenario'])
sexism_df = pd.read_excel("Sexism_Examples.xlsx")

In [4]:
aita_df

,VERDICT,Unnamed: 1,Unnamed: 2,Unnamed: 3,Raw reddit scenario,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,GPT-shortened Scenario,GPT-explanation,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,shortening GPT-prompt (to put in appendix of the paper)
0,not the asshole,20582.0,14ila0h,AITA for calling out people for not having the...,I love being outdoors and I play disc golf qui...,1.687697e+09,/r/AmItheAsshole/comments/14ila0h/aita_for_cal...,81.0,not the asshole,14.0,I play disc golf in a public park where leash ...,NaN,NaN,NaN,NaN,NaN,I am running an online experiment about subjec...
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,"The last several weeks I've gone, almost every...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,"I actually do like dogs, but I was attacked as...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,Friday afternoon I was playing with some frien...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,Am I the asshole? Am I out of place for callin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,no assholes here,1203.0,ylz5n4,AITA not helping other people in my class?,"In my university, grade boundaries for exams, ...",1.667569e+09,/r/AmItheAsshole/comments/ylz5n4/aita_not_help...,4.0,no assholes here,7.0,"My university grades on a curve, so if others ...",NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
sexism_df

,Scenario,Verdict,GPT explanation
0,The massive amount of people who think it's a ...,Sexist,"Yes, this scenario is sexist. It imposes a tra..."
1,When I’m the only woman with a group of men an...,depends,"Yes, this scenario can be considered sexist. B..."
2,"The statement ""women should dress modestly bec...",sexist,"Yes, this scenario is sexist. It places the re..."
3,"Using ""guys"" to address or describe a group th...",not sexist,"Using ""guys"" to address a mixed-gender group i..."
4,"Last week, I dyed my hair from my natural brun...",depends,"Yes, this scenario is sexist. The girl’s comme..."
5,"For example, my brother and I were watching a ...",sexist,"Yes, this scenario is sexist. The joke reinfor..."
6,My partner and I took his daughter shopping fo...,not sexist,"Yes, this scenario can be considered subtly se..."
7,I work in a large marketing department and rec...,sexist,"Yes, this scenario is sexist. The statement ma..."
8,I return to work from maternity leave (working...,sexist,"Yes, this scenario is sexist. The boss makes a..."
9,men insisting that men have better friendships...,sexist,"**Yes, this scenario is sexist.** It makes bro..."


In [6]:
scenarios_aita_list = aita_df['Raw reddit scenario'].tolist()
title_aita_list = aita_df['Unnamed: 3'].tolist()
scenarios_sexism_list = sexism_df['Scenario'].tolist()

In [7]:
aita_cleaned_scenarios = []

cur_scenario = ""
cur_title = ""
for title, scenario in zip(title_aita_list, scenarios_aita_list):
    # Check if we have a new title (not NaN)
    if not pd.isna(title):
        # Save previous scenario if it exists
        if cur_scenario != "":
            print(f"Completed Title: {cur_title}")
            aita_cleaned_scenarios.append(cur_scenario)
            print(f"Added Scenario: {cur_scenario}")
        
        # Start new scenario
        cur_title = title
        cur_scenario = str(scenario) if not pd.isna(scenario) else ""
        print(f"New Title: {cur_title}")
        print(f"New Scenario: {cur_scenario}")
    else:
        # Continuation of current scenario (title is NaN)
        if not pd.isna(scenario):
            if cur_scenario != "":
                cur_scenario += " " + str(scenario)
            else:
                cur_scenario = str(scenario)
            print(f"Updated Scenario: {cur_scenario}")

# Don't forget the last scenario
if cur_scenario != "":
    aita_cleaned_scenarios.append(cur_scenario)
    print(f"Final scenario added: {cur_scenario}")

New Title: AITA for calling out people for not having their dogs on a leash every time I'm at a park where it's mandatory?
New Scenario: I love being outdoors and I play disc golf quite regularly. A lot of disc golf courses are in multi-use public parks. One of my favorites near me is in this giant public park with baseball/softball fields, a swimming pool, basketball court, kids playgrounds, running trails, and the course is woven between all those things.
Updated Scenario: I love being outdoors and I play disc golf quite regularly. A lot of disc golf courses are in multi-use public parks. One of my favorites near me is in this giant public park with baseball/softball fields, a swimming pool, basketball court, kids playgrounds, running trails, and the course is woven between all those things. The last several weeks I've gone, almost every time I've ended up telling people they need to put their dog on a leash because it's mandatory. People tend to get really upset and try to argue wit

In [8]:
aita_cleaned_scenarios

['I love being outdoors and I play disc golf quite regularly. A lot of disc golf courses are in multi-use public parks. One of my favorites near me is in this giant public park with baseball/softball fields, a swimming pool, basketball court, kids playgrounds, running trails, and the course is woven between all those things. The last several weeks I\'ve gone, almost every time I\'ve ended up telling people they need to put their dog on a leash because it\'s mandatory. People tend to get really upset and try to argue with me that their dog is nice, wouldn\'t ever hurt anybody, they have control, etc. I always say it\'s the rules and that I\'m going to call the city if they don\'t put their dog on a leash. I actually do like dogs, but I was attacked as a kid by a loose dog. I lost one of my cats growing up to 2 loose dogs. My nephew has been bitten twice by the same neighbor\'s dog before it got put down. I just personally believe that dog owners are not respectful towards others in a lo

In [9]:
AITA_explanations = []
Sexism_explanations = []

In [10]:
from openai import OpenAI
client = OpenAI(api_key=OPEN_AI_API_KEY)

In [11]:
for scenario in aita_cleaned_scenarios:
    response = client.responses.create(
        model="gpt-4.1",
        input=(f"Is the post author in this scenario, who is at fault? Why or why not? Please provide write a 3-5 sentence explanation? Just the conclusion/TLDR/final judgement part."
        f"\n\nScenario: {scenario}"),
    )
    AITA_explanations.append(response.output_text)

In [12]:
for scenario in scenarios_sexism_list:
    response = client.responses.create(
        model="gpt-4.1",
        input=(f"Is this scenario sexist? Why or why not? Please provide write a 3-5 sentence explanation? Just the conclusion/TLDR/final judgement part"
        f"\n\nScenario: {scenario}"),
    )
    Sexism_explanations.append(response.output_text)

In [13]:
AITA_explanations

['**Conclusion/TLDR:** The post author is *not* at fault in this scenario. They are simply asking others to follow the clearly posted, city-mandated leash laws for everyone’s safety—a reasonable and responsible action, especially given their past negative experiences and the mixed-use nature of the park. Advocating for enforcement of established rules protects all park users, including children and those who are afraid of or allergic to dogs. While their approach may come across as direct or insistent, the underlying principle is respect for the law and consideration for public safety, not personal animosity towards dog owners.',
 "**Conclusion/TLDR:**  \nYou are not necessarily at fault for choosing not to correct your classmates, especially given the competitive, curved grading system at your university. While it might be more collegial or generous to help, you are under no formal obligation to do so, and your decision is understandable given that others' higher scores could negative

In [14]:
Sexism_explanations

["Yes, this scenario is sexist. It places an unfair expectation on women based solely on their gender, disregarding their personal autonomy and choices. By assuming it is a woman's duty to have children, society reinforces restrictive gender roles and limits women's freedom to decide their own paths. This perpetuates inequality and undermines respect for individual rights.",
 'Yes, this scenario can be considered subtly sexist. By singling out the woman for an apology, it implies that women are more delicate or less able to handle strong language than men. This reinforces outdated gender stereotypes and treats the woman differently solely because of her gender, rather than as an equal member of the group.',
 "Yes, this scenario is sexist. The statement places the responsibility for men's thoughts and actions on women, implying that women must modify their behavior or appearance to accommodate male impulses. It perpetuates harmful stereotypes by suggesting men lack self-control and wome

In [15]:
# Clean the explanation lists by removing any NaN values
AITA_explanations_cleaned = [exp for exp in AITA_explanations if pd.notna(exp) and str(exp).strip() != '']
Sexism_explanations_cleaned = [exp for exp in Sexism_explanations if pd.notna(exp) and str(exp).strip() != '']

print(f"Original AITA explanations: {len(AITA_explanations)}")
print(f"Cleaned AITA explanations: {len(AITA_explanations_cleaned)}")
print(f"Original Sexism explanations: {len(Sexism_explanations)}")
print(f"Cleaned Sexism explanations: {len(Sexism_explanations_cleaned)}")

# Display first explanation from each cleaned list to verify
print("\nFirst AITA explanation:")
print(AITA_explanations_cleaned[0][:200] + "..." if len(AITA_explanations_cleaned) > 0 else "No valid explanations")

print("\nFirst Sexism explanation:")
print(Sexism_explanations_cleaned[0][:200] + "..." if len(Sexism_explanations_cleaned) > 0 else "No valid explanations")

Original AITA explanations: 5
Cleaned AITA explanations: 5
Original Sexism explanations: 10
Cleaned Sexism explanations: 10

First AITA explanation:
**Conclusion/TLDR:** The post author is *not* at fault in this scenario. They are simply asking others to follow the clearly posted, city-mandated leash laws for everyone’s safety—a reasonable and res...

First Sexism explanation:
Yes, this scenario is sexist. It places an unfair expectation on women based solely on their gender, disregarding their personal autonomy and choices. By assuming it is a woman's duty to have children...


In [16]:
# First, let's get the cleaned titles that correspond to our cleaned scenarios
aita_cleaned_titles = []
cur_title = ""
for title, scenario in zip(title_aita_list, scenarios_aita_list):
    if not pd.isna(title):
        if cur_title != "":
            aita_cleaned_titles.append(cur_title)
        cur_title = title
    # No else clause needed - we just continue building the current scenario

# Don't forget the last title
if cur_title != "":
    aita_cleaned_titles.append(cur_title)

print(f"Number of cleaned titles: {len(aita_cleaned_titles)}")
print(f"Number of cleaned scenarios: {len(aita_cleaned_scenarios)}")
print(f"Number of cleaned explanations: {len(AITA_explanations_cleaned)}")

# Save explanations to TXT files for easier reading
with open('AITA_explanations.txt', 'w', encoding='utf-8') as file:
    file.write("AITA EXPLANATIONS\n")
    file.write("=" * 50 + "\n\n")
    for i, (title, explanation) in enumerate(zip(aita_cleaned_titles, AITA_explanations_cleaned), 1):
        file.write(f"EXPLANATION {i}:\n")
        file.write("-" * 20 + "\n")
        file.write(f"TITLE: {title}\n\n")
        file.write(f"EXPLANATION:\n{explanation}\n\n")
        file.write("=" * 50 + "\n\n")

with open('Sexism_explanations.txt', 'w', encoding='utf-8') as file:
    file.write("SEXISM EXPLANATIONS\n")
    file.write("=" * 50 + "\n\n")
    for i, explanation in enumerate(Sexism_explanations_cleaned, 1):
        file.write(f"EXPLANATION {i}:\n")
        file.write("-" * 20 + "\n")
        file.write(explanation + "\n\n")
        file.write("=" * 50 + "\n\n")

print(f"Saved {len(AITA_explanations_cleaned)} AITA explanations with titles to 'AITA_explanations.txt'")
print(f"Saved {len(Sexism_explanations_cleaned)} Sexism explanations to 'Sexism_explanations.txt'")

# Also save as YAML for structured format
import yaml

# Save as YAML with titles for AITA
aita_yaml = {
    'AITA_explanations': [
        {
            'title': title,
            'explanation': explanation
        }
        for title, explanation in zip(aita_cleaned_titles, AITA_explanations_cleaned)
    ]
}

sexism_yaml = {'Sexism_explanations': Sexism_explanations_cleaned}

with open('AITA_explanations.yaml', 'w', encoding='utf-8') as file:
    yaml.dump(aita_yaml, file, default_flow_style=False, allow_unicode=True)

with open('Sexism_explanations.yaml', 'w', encoding='utf-8') as file:
    yaml.dump(sexism_yaml, file, default_flow_style=False, allow_unicode=True)

print(f"Also saved as YAML with titles: 'AITA_explanations.yaml' and 'Sexism_explanations.yaml'")

Number of cleaned titles: 5
Number of cleaned scenarios: 5
Number of cleaned explanations: 5
Saved 5 AITA explanations with titles to 'AITA_explanations.txt'
Saved 10 Sexism explanations to 'Sexism_explanations.txt'
Also saved as YAML with titles: 'AITA_explanations.yaml' and 'Sexism_explanations.yaml'


In [17]:
# Create final Sexism dataset  
print("\n📊 Creating Sexism Final Dataset...")

sexism_final_dataset = pd.DataFrame({
    'Scenario': scenarios_sexism_list,  # From original dataset
    'Verdict': sexism_df['Verdict'].tolist(),  # From original dataset Verdict column
    'GPT_Explanation': Sexism_explanations_cleaned  # Generated explanations from GPT-4.1
})

print(f"✅ Sexism Dataset Shape: {sexism_final_dataset.shape}")
print(f"✅ Sexism Columns: {list(sexism_final_dataset.columns)}")

# Data integrity check
print(f"\n🔍 Sexism Data Integrity Check:")
print(f"Missing values per column:")
print(sexism_final_dataset.isnull().sum())

print(f"\n📋 Sexism Dataset Preview:")
print(sexism_final_dataset.head(2))


📊 Creating Sexism Final Dataset...
✅ Sexism Dataset Shape: (10, 3)
✅ Sexism Columns: ['Scenario', 'Verdict', 'GPT_Explanation']

🔍 Sexism Data Integrity Check:
Missing values per column:
Scenario           0
Verdict            0
GPT_Explanation    0
dtype: int64

📋 Sexism Dataset Preview:
                                            Scenario  Verdict  \
0  The massive amount of people who think it's a ...   Sexist   
1  When I’m the only woman with a group of men an...  depends   

                                     GPT_Explanation  
0  Yes, this scenario is sexist. It places an unf...  
1  Yes, this scenario can be considered subtly se...  


In [18]:
sexism_final_dataset

,Scenario,Verdict,GPT_Explanation
0,The massive amount of people who think it's a ...,Sexist,"Yes, this scenario is sexist. It places an unf..."
1,When I’m the only woman with a group of men an...,depends,"Yes, this scenario can be considered subtly se..."
2,"The statement ""women should dress modestly bec...",sexist,"Yes, this scenario is sexist. The statement pl..."
3,"Using ""guys"" to address or describe a group th...",not sexist,"**Conclusion/TLDR:** \nUsing ""guys"" to addres..."
4,"Last week, I dyed my hair from my natural brun...",depends,"Yes, this scenario is sexist. The comment assu..."
5,"For example, my brother and I were watching a ...",sexist,"Yes, this scenario is sexist. The joke relies ..."
6,My partner and I took his daughter shopping fo...,not sexist,This scenario may reflect a subtle form of sex...
7,I work in a large marketing department and rec...,sexist,"Yes, this scenario is sexist. The remark made ..."
8,I return to work from maternity leave (working...,sexist,"Yes, this scenario is sexist. The boss’s comme..."
9,men insisting that men have better friendships...,sexist,"Yes, this scenario is sexist. It relies on neg..."


In [19]:
# Save the final Sexism dataset to CSV
sexism_final_dataset.to_csv('Sexism_Final_Dataset.csv', index=False)

In [20]:
# =============================================================================
# CREATE FINAL DATASETS - AFTER ALL EXPLANATIONS ARE GENERATED
# =============================================================================

print("🎯 Creating final datasets with all required information...")
print("=" * 70)

# Extract AITA verdicts and existing shortened scenarios from original dataset
aita_verdicts = []
aita_existing_shortened = []

for index, row in aita_df.iterrows():
    if pd.notna(row['VERDICT']):
        aita_verdicts.append(row['VERDICT'])
        if pd.notna(row['GPT-shortened Scenario']):
            aita_existing_shortened.append(row['GPT-shortened Scenario'])

print(f"✅ Found {len(aita_verdicts)} AITA verdicts")
print(f"✅ Found {len(aita_existing_shortened)} existing shortened scenarios")
print(f"✅ Using {len(aita_cleaned_titles)} cleaned titles")
print(f"✅ Using {len(aita_cleaned_scenarios)} cleaned scenarios")
print(f"✅ Using {len(AITA_explanations_cleaned)} generated explanations")

🎯 Creating final datasets with all required information...
✅ Found 5 AITA verdicts
✅ Found 5 existing shortened scenarios
✅ Using 5 cleaned titles
✅ Using 5 cleaned scenarios
✅ Using 5 generated explanations


In [21]:
# create final AITA dataset
aita_final_dataset = pd.DataFrame({
    'Title': aita_cleaned_titles,  # From cleaned titles
    'Scenario': aita_cleaned_scenarios,  # From cleaned scenarios
    'Verdict': aita_verdicts,  # From original dataset verdicts
    'Existing Shortened Scenario': aita_existing_shortened,  # From original dataset shortened
    'GPT_Explanation': AITA_explanations_cleaned  # Generated explanations from GPT-4.1
})

In [22]:
aita_final_dataset

,Title,Scenario,Verdict,Existing Shortened Scenario,GPT_Explanation
0,AITA for calling out people for not having the...,I love being outdoors and I play disc golf qui...,not the asshole,I play disc golf in a public park where leash ...,**Conclusion/TLDR:** The post author is *not* ...
1,AITA not helping other people in my class?,"In my university, grade boundaries for exams, ...",no assholes here,"My university grades on a curve, so if others ...",**Conclusion/TLDR:** \nYou are not necessaril...
2,AITA for causing an awkward argument at a dinn...,"So, this happened at a dinner party a few days...",everyone sucks,"At a dinner party, a guy I barely knew started...",**TL;DR:** While your concerns about misinform...
3,AITA for calling my sister selfish over brownies?,"Hey Reddit, I (16) recently had a disagreement...",asshole,My sister (14) baked brownies but got mad when...,**TL;DR / Final Judgement:** The post author (...
4,AITA for locking my bedroom door?,I F14 live with my mom F39 and my grandma F66....,everyone sucks,I (14F) stayed home from school after overslee...,"**Conclusion/TL;DR:** \nYou, the post author,..."


In [23]:
# Save the final AITA dataset to CSV
aita_final_dataset.to_csv('AITA_Final_Dataset.csv', index=False)